# Quantum Cryptography : BB84 protocol 

**[BB84](https://en.wikipedia.org/wiki/BB84)** protocol was set up in 1984 by one of our famous IBM'ers Charles Bennett with his colleague Gilles Brassard. 
It has been experimented a few years later in the first demonstration of quantum key distribution  : [quantum key distribution](https://en.wikipedia.org/wiki/Quantum_key_distribution) by Charles Bennett and John Smolin in IBM [C. H. Bennett, F. Bessette, G. Brassard, L. Salvail, and J. Smolin, J. of Cryptography **5**, 3 (1992) ]. Charles and John are still part of the quantum IBM team. 

<img src="charlie_john_qkd.jpg" width="500"/>
<center>Charles Bennett and John Smolin at T.J. Watson IBM research center.</center>

BB84 protocol allows to communicate a cryptographie key from one point to another and knowing that it has not been compromised. 


## BB84 Protocol 

The steps are the following : 

1. Alice chooses two random bit strings of length  $n$ : $k$ and $b$. The list $k$ has the key value. The list $b$ represents the bases choice for Alice to encode the bits of $k$. When  $b_i=0$ (meaning if the  $i^{th}$ base bit is zero ), she encodes the $i^{th}$ qubit in the standard base $\{|0\rangle, |1\rangle \}$, and if $b_i=1$, she uses the base  $\{|+\rangle, |-\rangle \}$, where $|+\rangle:=\frac{1}{\sqrt{2}}(|0\rangle +|1\rangle)$, $|-\rangle:=\frac{1}{\sqrt{2}}(|0\rangle -|1\rangle)$. 

This can be showned as follows : 


<img src="encoding_Alice.png" alt="drawing" width="300"/>

2. After endoding $n$ bits, Alice sends them to Bob. He chooses a random bit string $\tilde{b}$ of length $n$ for the measurement bases he will be using.. Bob records his measurement results  $\tilde{k_i}$ along with the bases used $\tilde{b_i}$, in an array.

3. Then Alice and Bob compare their bases $b_i$ and $\tilde{b}_i$. Whenever  $b_i \neq \tilde{b}_i$, Bob did not measure in the same base Alice did encode. The probability of having measured the correct value for $k_i$ is $\frac{1}{2}$, in this case this position is discarded. Howeever, if $b_i = \tilde{b}_i$, then the qubit was preepared and measured in the same basis (and if noboby did spy on the communication) Bob did measure a correct value : $\tilde{k}_i = k_i$. These values make the key 

## Example : 

Let's assume Alice random key value is :  $k=`0111001`$ and her bases choice : $b=`1101000`$, and let's assume Bob's bases are : $\tilde{b}=`1001101`$. Look at the values below and note that when bases eare not hte same Bob has one chance out of two to get a correct measurement. 
<img src="example_bb84.png" alt="drawing" width="500"/>

In this case, the key is '0110', and to know if it has been seen by Eve, Alice and Bob will have to sacrifice some of theses bits. If a measurement was made during the distribution, le qubit state would have change with probability $\frac{1}{4}$. By verifying $m$ bits values, the probability of not detecting Eve decays as $\left(\frac{3}{4}\right)^m$. So if the verify enough bits, they will gain confidence that Eve did not spy. 

### Message encryption

One the key is known and secretely has been secretly distributed, Alice and Bob can use it very easily  :  [Masque Jetable](https://fr.wikipedia.org/wiki/Masque_jetable) : Alice just adds bitwise modulo 2 the two lists.  (in other words she performs bitwise XOR). With the above example, key is $\text{key}=`0110`$. Secret message is $m=`1100`$, so encrypted message is : $c=m\oplus \text{key (mod 2)} = `1010`$. Bob can decipher in doing the same thing : add the key to the message : $m=c\oplus \text{key (mod 2)}$.

## workshop : 

In this notebook you will be in Bob's role, and with Alice you will generate a key unsing BB84 protocol, then you will beable to decodee Alice's message.

---

There will be 4 steps : 

1. Alice prepares her key and bases. The function (provided) `alice_prepare_qubit` is used to send to you the qubits states. 

2. Bob (you) measure the recevied qubit states in your basis list, you will have to code the function  `build_bob_measure_circuit`. This function receives an index to point in the qubits and in Bob's basis list as well as in the quantum circuit list (which has prepared Alice's qubit state). The function returns a quantum circuit which can be appended to Alice's circuit used to prepare her qubit. Bob circuit simply performs a measure about Bob's base, for each index position. 

3. Then you will have to build the code for making the key Alice and Bob will agree on (by discarding the values that have been emitted and received on non identical basis).

4. Finally, we will be able to write the function to decipher Alice's massage with the key. You generate a bit string, which is a morse code text, you will need to make it readable. 

---

This workhop is derived from the IBM Quantum Computing Challenge, on may 4th, 2020. 

### Setup

In [1]:
# import libraries
import BB84HO
from random import randint
from qiskit import QuantumCircuit, execute, Aer
backend = Aer.get_backend('qasm_simulator')

bases_length = 1500

### Alice prepares her key and her bases

In [2]:
# clef aléatoire
alice_key = ""
for i in range(bases_length): 
    alice_key += str(randint(0,1))

# bases aléatoires
alice_bases = ""
for i in range(bases_length):
    if randint(0,1):
        alice_bases += "z"
    else:
        alice_bases += "x"

### Bob prepares his bases : 

In [3]:
bob_bases = ""
for i in range(bases_length):
    if randint(0,1):
        bob_bases += "z"
    else:
        bob_bases += "x"
        
print('Bob\'s bases :', bob_bases)

Bob's bases : xzzzxzxxxzzzxzxxzxxzzxzxzzzzxxzzzxxxzxzzxzzzzxxxzzzxzxzzxzxxxzzxzxzzxxzxzxzxzzzzzxzzzxxzxxxzzzxzzxxzxzxxxxzzzzzxxzzxzzxxxxzxxxxxxzzzxzzzzzxxxxzzxxxzxzxxxzzzzzxzzzzzzzxzzzxzxxxxzzzxzxxzxxxxxzzxxzzzzxxxxzxxzxxxxzzxzzzxxxzzzxxxzzzxxxzzxxxzxzzzzzxzxzxxzzxxxxxzxxxxxxxzzxzzzzzzxxxzzxzzxxzxxzxzxzxzzzxxzzxzxxzxxzzxxzzxxxxxxzxxzzxxzzzxxxzzxxxzzzzxxzzxxxzxxxxxxxzzxzxzzxxzxzxzxzxxxxzzxxzxzzxxzzxxzxzxxxxxxzzzzxxxzxzzxzzxxxxxzxzzxxzzxzxzzzxxxxxxxxxxxxxxxxxxzxxxxzzxzxzzxzxxxzxzxxxxxzzzzzzxzzxxzxxzxzxxzzzxxxxxzzzzzxzxzzxxxxzxxxzzxzxxxxzzzzzzzzzxzxzzzxxzzzxxxxzxzzzzxzxzzzxxzxxzxzzxzxxzxzzxzzxxzzzzxxzxzxxxxxzxzxzxxxxxxxzzxxxxzzzxxxzzxzzzxxxzzzxzxxxzzzzxxxxxxzzxzxzxzzxzxzxzxxxxxxxzzxxxzxzxzxxzzzxzzxxzxzxzzxxxxxxzxxxzzxzxzxxxxxzzzxxxxxxzzzxzxzxxzxzxzxzzxzzxzzzzzxxzzxzzzzzxxxzxxzzzxzzzxxxxxzzzxxzxzxxxxxzzxzxxzzxxzxxxxzxxxzxxzxzzxzxxxzzxxzxxxzzzzxzzzzxzxxzzxxxzzzxzxzzxxxzzxxxzzzxxxzzzxzzxzxzzzzzzzzxzxzzzxxxzzxzzzzzxzxxzzxxzzzxxxzxxxxxxxxxzzxzzzxxzzzzxxxzzzxzxzzxxxxxzzzxzzzzzxxzxzzxxzzzxxzxzzx

### Now Bob performs the measurements (with his bases) on states prepared by Alice (with her key and bases): 


In [4]:
bob_results = ""

def bb84():
    print('Bob\'s bases:', bob_bases)

    # Maintenant Alice prépare ses qubits, un par un en utilisant ses bases, 
    # Bob les mesure, en utilisant ses propres bases.
    bob_results = ""
    
    # pour chaque bit
    for index in range(bases_length):
        # Alice crée l'état de son qubit
        thisqubit_circuit = BB84HO.prepare_alice_qubit(index, alice_bases, alice_key)
        
        # Bob prépare la lecture (c'est la fonction à coder ci-dessous) 
        bob_measure = build_bob_measure_circuit(index, bob_bases, thisqubit_circuit)
        
        # maintenant on utilise le simulateur pour éxecuter le circuit et on effectue
        # la mesure
        bob_job = execute(bob_measure, backend=backend, shots=1)
        bob_reading = bob_job.result()
        
        # pour chaque résultat (dict, on le transforme en liste et on prend le premier élément : 
        # la valeur de mesure pour le qubit
        bob_results += list(bob_reading.get_counts(bob_measure))[0]
    return bob_results

# Ecrire cette fonction : 
# pour compléter un circuit utilisant 1 qubit : qc envoyé par Alice de manière à 
# pour effectuer la mesure en utilisant l'orientation de la base à l'index "index" de Bob.
# La fonction retourne le circuit ainsi construit.
def build_bob_measure_circuit(index, bob_bases, qc):    
    if bob_bases[index] == "x":
        qc.h([0])
    elif bob_bases[index] == "z":
        pass
    qc.measure(0,0)
    return qc

bob_bits = bb84()        
print('Bob\'s bits: ', bob_bits)


Bob's bases: xzzzxzxxxzzzxzxxzxxzzxzxzzzzxxzzzxxxzxzzxzzzzxxxzzzxzxzzxzxxxzzxzxzzxxzxzxzxzzzzzxzzzxxzxxxzzzxzzxxzxzxxxxzzzzzxxzzxzzxxxxzxxxxxxzzzxzzzzzxxxxzzxxxzxzxxxzzzzzxzzzzzzzxzzzxzxxxxzzzxzxxzxxxxxzzxxzzzzxxxxzxxzxxxxzzxzzzxxxzzzxxxzzzxxxzzxxxzxzzzzzxzxzxxzzxxxxxzxxxxxxxzzxzzzzzzxxxzzxzzxxzxxzxzxzxzzzxxzzxzxxzxxzzxxzzxxxxxxzxxzzxxzzzxxxzzxxxzzzzxxzzxxxzxxxxxxxzzxzxzzxxzxzxzxzxxxxzzxxzxzzxxzzxxzxzxxxxxxzzzzxxxzxzzxzzxxxxxzxzzxxzzxzxzzzxxxxxxxxxxxxxxxxxxzxxxxzzxzxzzxzxxxzxzxxxxxzzzzzzxzzxxzxxzxzxxzzzxxxxxzzzzzxzxzzxxxxzxxxzzxzxxxxzzzzzzzzzxzxzzzxxzzzxxxxzxzzzzxzxzzzxxzxxzxzzxzxxzxzzxzzxxzzzzxxzxzxxxxxzxzxzxxxxxxxzzxxxxzzzxxxzzxzzzxxxzzzxzxxxzzzzxxxxxxzzxzxzxzzxzxzxzxxxxxxxzzxxxzxzxzxxzzzxzzxxzxzxzzxxxxxxzxxxzzxzxzxxxxxzzzxxxxxxzzzxzxzxxzxzxzxzzxzzxzzzzzxxzzxzzzzzxxxzxxzzzxzzzxxxxxzzzxxzxzxxxxxzzxzxxzzxxzxxxxzxxxzxxzxzzxzxxxzzxxzxxxzzzzxzzzzxzxxzzxxxzzzxzxzzxxxzzxxxzzzxxxzzzxzzxzxzzzzzzzzxzxzzzxxxzzxzzzzzxzxxzzxxzzzxxxzxxxxxxxxxzzxzzzxxzzzzxxxzzzxzxzzxxxxxzzzxzzzzzxxzxzzxxzzzxxzxzzxz

### After Bob made his measurement, Alice and Bob share their bases: 


In [5]:
print("alice_bases = ", alice_bases, "\n")
print("bob_bases   = ", bob_bases)
# l'échange des bases peut aussi avoir été espionné, et dans ce cas, si Eve a aussi
# mesuré les qubits, elle peut reconstituer la clef. Mais Alice et Bob sauront qu'ils 
# ont été écoutés et devront changer leur canal de communication.

alice_bases =  xxzzzxxzzzzxzxzxxxzxxzxxxxxzxzxxzzzxzzzzxzxzxzzzxzxxzxxzxzxxxxzxxxxzzxxxxxxxxzzxxxxxzzzzxxxxzzzzxxzzxzzzzzzzxxxxxxxxzxxzxzzzxzxxzxxxxzzzxzzzxxxzzzxxzxxxxxxxxzzxzxzxxzzzzzxxxxxzzxzxxxzzxzzzzxzzxxxxxzzxzxxxzzxzzxxxzzzzxxzzxxxzxzzzxzxxxxxxzxzxzzxxxxxzzxxzxzxxzxxxzzzxxzxxxzxzzzxxxzzzxxzxzzxxxzzxzzxzxxxxzzxzxzxxzzxxxxzxxxxxxxxzzxxzxxzzxxzxzxxxxxzzzzzzxxzzzzzzzxxxxxxxzzxzzzzzzxxzxxxzzxxzzzzzxzxxzxxxzzzzxxzxxxzzzzzzxzzxzxxxxzzxxzxxxzxzxxzzxxzxzxzzxzxxzxzzxxxxzzzzzxzxxzxxxzxxxxxxzxxzxzxxxzxxxxxzxxzxxzxzxzxxzzxzzxzzzzxxzzxzzxzzzzzzxzzxxzxxzzxzxxzxzxxzzzxzzzzxzxzxzxxxxzxzxxzxxzzzzxxzzxxzxxzxzxzzxxzxxxzzzzxxzzzxzxzzxxzxzxxxzzxzxzxxxzxzzzzxxxzxzxzxxxxzzzzxxzzxzzxxzxxxxzxzzxzxzxxxzxzzxzxxzxxxzzzxzxxzzxxzzxxxxxzxxzxxzxxxxxzxxxxzzxxxxzxzxxxzxzxzxxxxxzzxzxxxzxzxzzzzzxxxzxzzxzzzxxxzxxzxzxzzzxxzzxzxxzxxxzxzzzxzzxxxxxzxzxzxxzxxxxzzzxzzxxxxzxxxzxzzxzzxxzxzxzxzzzzzxzzzxzzxzxzzzxxzxxxzzxxzxxzzxxxxxzxxxxxxxzzxzxxzxzxxzzxxzxzxxzxxzxxxxzxzxzzzzzzxzxzxxxzzzxzxxxzzxzzxxxzzzzxzxxxxxzzzxxxxxzxxxzzxx

### Now Alice and Bob can construct their key :
They will keep only the bits corresponding to index where the bases were identical. 


In [6]:
key = ""
#
for i in range(bases_length):
    if bob_bases[i] == alice_bases[i]: 
        key += bob_bits[i]
print("alice_key =", alice_key)
print("alice_bases = ", alice_bases)
print("bob_bases = ", bob_bases)
print("bot_bits =", bob_bits)

print("key = ", key)
print("longueur de la clef", len(key))


alice_key = 0111001010100101011000111000011001001010111011111110101010001110001010110001110000111101011100100100100100010101101111010001011100001010001110000001100000011100001100110111001001111001010100000111101111100111010010001100011010110001010111111101100101101011101110100110100000011000101000101110011001011001111111010110001100110100010101111100010010011010100001010110000001110110100011001000100010000111101010101111011000000001110001001100111001010101011000001101001110010000010010110110110101101101000010001100100010001001110111101100111100010010100010100011000000001100000110011101011000010101110010010011100111110011011100111101110111000101000001111001000100110011011101101010011010000110111111011001001101011011101111110111110110000001100101000010001110110010010010011000011110001110000101111000111110100101101001001000000101001111010000001011001110010001111111111111100001010101001110101111000110000111111100101001000000110101001000111001101101011000111001110100101001010100110111100001

### Now Alice can use the key to encode her message : 

execute the following cell

In [7]:
mess = BB84HO.code(BB84HO.alice_message_clair, key)

print(mess)
print(len(mess))

101011101010101111100001001010010011101000001011110010110110111100010000010111110010100110110111110011101000100010110111000011101100111000111011011001110011101001111001100100000010000001110110010111011010100100000001100000001100110100100110110001100101101000000111100010100111111001111010001101000100110001000010000110010000101110101101000111000011101001010011011100010001110010010101001000110101000110000011011010101110001000111
429


### Function to decode the message using the key: 

In [8]:
#mess = BB84HO.alice_message  # ceci est le message encrypté par Alice, avec la clef
print("message :", mess)
print(len(mess))

def decode(message,clef):
    # 1 on vérifie que la clef est assez longue.
    # 2 on calcule le ou exclusif bit à bit dans un nouveau string. 
    if len(message) > len(clef):
        print("la clef est trop courte, recommencez")
        return
    else: 
        clef = clef[0:len(message)]
        print("clef :", clef)
        clair = ""
        for i in range(len(message)):
            clair += str((int(clef[i]) + int(message[i])) % 2)
        return clair
   
    
clair = decode(mess,key)
clair = clair.rstrip("0")
print("clair   :" ,clair)

message : 101011101010101111100001001010010011101000001011110010110110111100010000010111110010100110110111110011101000100010110111000011101100111000111011011001110011101001111001100100000010000001110110010111011010100100000001100000001100110100100110110001100101101000000111100010100111111001111010001101000100110001000010000110010000101110101101000111000011101001010011011100010001110010010101001000110101000110000011011010101110001000111
429
clef : 011101111000011011010100100011000010110011011000101001111100001110100100000010101011001011010001100101001100110100101110100111011000001010011101101111101001110010101100001000011000110101010000011100010110000001101011110100101001011100000011100010100011011010110101000100000001101010101000111110010111110111010000110010100110111010011001101100010111011111111000000100100100100000100001100100011111110101011000011100000011010011111
clair   : 1101100100101101001101011010010100010110110100110110110010101100101101000101010110011011011001100101101001000

Message is in morse code :

- point : '1' ("ti")
- dash : '11' ("ta")
- separator : '0'
- letter separator :  '00'
- word separator: '000'.

For example :
<img src="qiskit_morse.png" alt="drawing" width="500"/>

### Try and decode Alice message  !

You may "split" the string into a word list `texte.split("000")`, then split words into letters with `mot.split("00")`, finally split letters into signs ("ti" and "ta"), then pick up each letter from the provided dictionnary to display the message. 


In [24]:
M = { '.-':'a', '-...':'b', '-.-.':'c', '-..':'d', '.':'e', '..-.':'f', '--.':'g', 
     '....':'h', '..':'i', '.---':'j', '-.-':'k', '.-..':'l', '--':'m', '-.':'n',
     '---':'o', '.--.':'p', '--.-':'q', '.-.':'r', '...':'s', '-':'t', '..-':'u',
     '...-':'v', '.--':'w', '-..-':'x', '-.--':'y', '--..':'z', '.----':'1', 
     '..---':'2', '...--':'3', '....-':'4', '.....':'5', '-....':'6', '--...':'7',
     '---..':'8', '----.':'9', '-----':'0', '--..--':',', '.-.-.-':'.', '..--..':'?',
     '-..-.':'/', '-....-':'-', '--..--':',', '---...':':', '-.--.':'(', '-.--.-':')',
     '..-..':'é','.--.-':'à', '-.-.--':'!'} 

mots = []
lettres = []
signes = []

phrase_clair = ""
mots = clair.split('000')
for mot in mots:
    mot_clair = " "
    lettres = mot.split('00')
    for lettre in lettres:
        lettre_morse = ""
        signes = lettre.split('0')
        for signe in signes:
            if signe == '11':
                lettre_morse += "-"
            elif signe == '1':
                lettre_morse += "."
        lettre_clair = M.get(lettre_morse,'*')
        mot_clair += lettre_clair
    phrase_clair += mot_clair
print(phrase_clair)


 merci pour votre attention, cet atelier est maintenant terminé, bravo !


In [ ]:
def code_morse(text):
    morse_output = ""
    mots = text.split(' ')
    for s in mots:
        for c in s: 
            morse = MORSE_DICT.get(c)
            for m in morse:
                if m == '.':
                    morse_output += '10'
                elif m == "-":
                    morse_output += '110'
            morse_output += "0"
        morse_output += "0"
    return morse_output

m = code_morse("merci pour votre attention, cet atelier est maintenant terminé, bravo !")
print(m)